In [ ]:

```js
/* [GET] / */
app.get("/", (req, res) => {
    res.send("Home Page");
});

//...
```

**Vấn đề**: 
- Nếu cấu hình viết API như trên thì tương lai file `server.js` sẽ rất dài và khó quản lý
- Trong ứng dụng sẽ có nhiều route:
	- users 
	- categories
	- products 
	- posts
	- comments
	- ...

- Mà mỗi một tài nguyên lại viết REST API thì ít nhất có các thao tác cơ bản CRUD => sinh ra tầm khoảng 5 route nữa:
	- Lấy danh sách
	- Lấy chi tiết
	- Thêm
	- Sửa
	- Xoá
	...

> Học cách chia ra

In [ ]:
// Bước 1: Tạo thư mục src/routes/index.js
// Đọc file...

In [ ]:

// Bước 2: Chuyển hết route từ server.js vào file này:
// Dựa vào tài nguyên để đặt tên route
// VD: /api/tasks --> Tên file: tasks.route.js
// Sử dụng express.Router() để chia nhỏ các route ra thành nhiều cấp bậc khác nhau

/* 
Vấn đề: Ở file server.js. Khi chúng ta viết route nhiều cấp bậc:
- /api/tasks
- /api/tasks/1
- /api/tasks/1/abc
- ...
> Hậu quả: Lặp đi lặp lại /api/tasks. Nếu sửa thì sẽ sửa tất

* Giải pháp: Tại file task.route.js chúng ta chỉ tập trung vào cấp sau cùng:
- /api/tasks            --> /
- /api/tasks/:id        --> /1
- /api/tasks/:id/:type  --> /1/abc
*/

const express = require("express");
const router = express.Router();

/* Database */
const db = {
    tasks: [
        {
            id: 1,
            title: "Nấu cơm",
        },
        {
            id: 2,
            title: "Rửa bát",
        },
    ],
};

// Thay thế app.get.. = router.get...
/* [GET] /api/tasks */
router.get("/", (req, res) => {
    res.json(db.tasks);
});

/* 
- [GET] /api/tasks/1 
*/
router.get("/:id", (req, res) => {
    console.log(req.params.id); // 1
    console.log(req.query.name); // NguyenMinhViet

    res.json({
        data: "Detail",
    });
});

/* [POST] /api/tasks */
router.post("/", (req, res) => {
    console.log(req.body);

    const maxId = Math.max(...db.tasks.map((t) => t.id));
    const newTask = {
        id: maxId + 1,
        title: req.body.title,
    };
    db.tasks.push(newTask);
    res.json(db.tasks);
});

module.exports = router;


In [ ]:
/* Khai báo */
const express = require("express");
const app = express();
const port = 3001;

/* Middleware */
app.use(express.json());

/* [GET] / */
app.get("/", (req, res) => {
    res.send("Home Page");
});

/* Listen */
app.listen(port, () => {
    console.log(`Đang lắng nghe tại cổng: ${port}`);
});


In [ ]:
// index.js

/* Vai trò của file này: Tạo một router con tiếp. Nhưng ở cấp cao hơn và import từng route nhỏ hơn */
const express = require("express");
const router = express.Router();
const tasksRouter = require("./tasks.route");

// .all(): Không phân biệt method
// Có nghĩa là: nó sẽ nối /tasks với tasksRouter
router.all("/tasks", tasksRouter);

module.exports = router;


In [ ]:
// server.js

/* Khai báo */
const express = require("express");
// Import từ index.js
const apiRouter = require("./src/routes");

//...

// Nối thêm cấp route
app.all("/api", apiRouter);


//...


In [ ]:
// Tương tự với bài post
// posts.route.js

const express = require("express");
const router = express.Router();

/* Database */
const db = {
    posts: [
        {
            id: 1,
            title: "Bài đăng 1",
        },
        {
            id: 2,
            title: "Bài đăng 2",
        },
    ],
};

/* [GET] /api/posts */
router.get("/", (req, res) => {
    res.json(db.posts);
});

/* 
- [GET] /api/posts/1 
*/
router.get("/:id", (req, res) => {
    res.json({
        data: "Detail",
    });
});

/* [POST] /api/posts */
router.post("/", (req, res) => {
    console.log(req.body);

    const maxId = Math.max(...db.posts.map((t) => t.id));
    const newTask = {
        id: maxId + 1,
        title: req.body.title,
    };
    db.posts.push(newTask);
    res.json(db.posts);
});

module.exports = router;


In [ ]:
const express = require("express");
const router = express.Router();
const tasksRouter = require("./tasks.route");
// Thêm postsRouter
const postsRouter = require("./posts.route");

router.use("/tasks", tasksRouter);

// Thêm route
router.use("/posts", postsRouter);

module.exports = router;


In [ ]:
// Tối ưu file index.js bằng cách đọc file

const express = require("express");
const router = express.Router();
const tasksRouter = require("./tasks.route");
// Thêm postsRouter --> Mất công thêm
const postsRouter = require("./posts.route");

router.use("/tasks", tasksRouter);

// Thêm route --> Mất công thêm
router.use("/posts", postsRouter);

module.exports = router;

// Ý tưởng: Viết ra tiện ích "đọc file"
// - Đọc tất cả file ở thư mục src/route (Ngoại trừ file index.js)
// - Nạp những file đó vào và đưa vào use() tương ứng

In [ ]:
const express = require("express");
const fs = require("node:fs");
const router = express.Router();

const basePath = "./src/routes";
const postFix = ".route.js"; // Hậu tố

fs.readdirSync(basePath)
    .filter((_fileName) => _fileName.endsWith(postFix)) // tasks.route.js
    .forEach((fileName) => {
        const resource = fileName.replace(postFix, ""); // tasks
        router.use(`/${resource}`, require(`./${fileName}`));
    });

// const tasksRouter = require("./tasks.route");
// const postsRouter = require("./posts.route");

// router.use("/tasks", tasksRouter);
// router.use("/posts", postsRouter);

module.exports = router;
